## Cython

In this notebook, we will use the `cythonmagic` extension, to demonstrate why and how to use cython.

Note that this is not the typical usage pattern for cython: When we are done going through this notebook, we will also look at how to use cython in the context of modules and libraries.

But for now, let's load the `Cython` IPython extension. This allows us to mark cells as cython cells by starting them with `%%cython`. 

In [14]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


First, let's see what this is good for.

Consider a very simple function defined in python:

In [15]:
def my_poly(a,b):
    return 10.5 * a + 3 * (b**2)

We can use the `timeit` IPython magic to tell us how much time this function takes:

In [16]:
time_my_poly = %timeit -o my_poly(10, 2)

The slowest run took 5.82 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 504 ns per loop


### Question: what's taking it so long? 

The equivalent cython function is defined below in a `%%cython` cell. Note that the only difference is that we tell the function to treat these variables as double-precision numbers. 

**Important**: If this code were written in a regular python cell it would produce a syntax error. Cython is a 'dialect' of python, but it is not exactly like python. In fact, cython is a proper superset of python. That means that any python code is legitimate cython code, but the opposite. We will see one way to deal with this issue in a little bit.

In [17]:
%%cython
def my_polyx(double a, double b):
    return 10.5 * a + 3 * (b**2)

In [18]:
time_my_polyx = %timeit -o my_polyx(10, 2)

The slowest run took 15.04 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 144 ns per loop


In [21]:
speedup = time_my_poly.best/time_my_polyx.best
print(speedup)

3.503076047182103


This shows that we can gain more than a 3-fold speedup for even a trivial piece of code.

Let's consider an (only slightly) more interesting example, the calculation of the fibonacci series that we considered for the profiling examples:

In [22]:
def fib(n):
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a

    return a

The first step in creating the cython version of this is to copy the code into a cell and decorate it with `%%cython`

In [24]:
%%cython
def fibx(int n):
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a
    return a

In [25]:
time_fib = %timeit -o fib(10)
time_fibx = %timeit -o fibx(10)
print(time_fib.best/time_fibx.best)

1000000 loops, best of 3: 1.02 µs per loop
The slowest run took 5.25 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 209 ns per loop
4.867633966165363


As before, this results in some speedup

Next, we add minimal type information we will use the `cdef` keyword (a cython keyword!) to define local variables (integers in this case)

In [26]:
%%cython
def fibx(int n):
    cdef int i, a, b
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a
    return a

In [27]:
time_fib = %timeit -o fib(10)
time_fibx = %timeit -o fibx(10)
print(time_fib.best/time_fibx.best)

1000000 loops, best of 3: 1.01 µs per loop
The slowest run took 20.68 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 63.3 ns per loop
15.901533046104458


Resolving types of the variables inside of the function and a "promise" to use them only as integers results in a more than 10-fold speedup!

Let's pause to consider the implications of this. The C code required to perform the same calculation as `fibx` might look something like this: 

        int fib(int n){
            int tmp, i, a, b;
            a = b = 1;
            for(i=0; i<n; i++){
                 tmp = a;
                 a += b;
                 b = tmp;}	     
            return a;}

In and of itself, that's not too terrible, but can get unpleasant if you write more than this trivial function. The main issue is that integrating this code into a python program is not trivial and requires writing extension code (see [this blog post](http://people.duke.edu/~ccc14/sta-663/Optimization_Bakeoff.html) for details...). This also has overhead that is hard to optimize. Cython writes highly optimized python extension code, making it easy to separate out performance bottle-necks and compile them, but keep using the functions in your python code. 

## Optional section: writing cython that also works as python

Remember that we mentioned that cython code is not always syntactically correct python code? Sometimes you might want to write code that can be compiled as cython, but would also work as python, as a fall-back (e.g., when you don't have a compiler, or some-such). If you want to do that, you can use the cython API. The following cell is a simple example. This can be switched between (un-compiled) python and (compiled) cython by adding/removing the `%%cython` cell magic command at the top of the cell.

In [31]:
import cython
@cython.locals(n=cython.int)
def fib_pure_python(n):
    cython.declare(a=cython.int,
                   b=cython.int,
                   i=cython.int)
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a
    return a


In [32]:
%%cython
import cython
@cython.locals(n=cython.int)
def fib_pure_pythonx(n):
    cython.declare(a=cython.int,
                   b=cython.int,
                   i=cython.int)
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a
    return a


In [34]:
time_fib_pure_python = %timeit -o fib_pure_python(10)
time_fib_pure_pythonx = %timeit -o fib_pure_pythonx(10)
print(time_fib_pure_python.best/time_fib_pure_pythonx.best)

The slowest run took 5.73 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 1.7 µs per loop
The slowest run took 20.49 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 54.4 ns per loop
31.263009706321906


### Question: why do you think this results in an even more dramatic speedup?